In [1]:
import requests
import datetime

class RiskAnalyzer:
    """Анализатор рисков для ИП и ЮЛ."""

    API_KEY = "FJJWoPhZ5uGUL1ve"
    BASE_URLS = {
        "ИП": "https://api.checko.ru/v2/entrepreneur",
        "ЮЛ": "https://api.checko.ru/v2/company"
    }
     
    
    
    
    CRITICAL_RISKS_UL = {
        "1. Ограничение доступа к сведениям (ФНС)",
        "2. Ликвидация компании",
        "4. Недостоверные сведения о руководителе",
        "5. Недостоверные сведения об учредителях",
        "6. Записи в реестре банкротств (ЕФРСБ)",
        "7. Включение в реестр недобросовестных поставщиков",
        "8. Дисквалифицированные лица в руководстве",
        "11. Массовый руководитель",
        "12. Массовый учредитель",
        "9. Включение в санкционные списки",
        "10. Нелегальная деятельность на финансовом рынке",
        "18. Компания существует менее 3 месяцев",
    }

    NON_CRITICAL_RISKS_UL = {
        "3. Недостоверные сведения о юридическом адресе",
        "15. Наличие задолженности по налогам",
        "17. Высокая кредитная нагрузка региона (Желтый уровень)",
        "19. Компания существует менее 1 года",
        "20. Нарушения при получении господдержки"
    }
    
    CRITICAL_RISKS_IP = {
        "1. Ограничение доступа к сведениям (ФНС)",
        "2. Прекращение деятельности",
        "4. Массовый руководитель",
        "5. Массовый учредитель",
        "3. Включение в реестр недобросовестных поставщиков",
        "6. Записи в реестре банкротств (ЕФРСБ)",
        "10. Компания существует менее 3 месяцев"
    }
    
    NON_CRITICAL_RISKS_IP = {
        "7. Нарушения при получении господдержки",
        "11. Компания существует менее 1 года"
    }

    HIGH_RISK_REGIONS = {
        "Красный": [
            "Карачаево-Черкесская Республика",
            "Республика Марий Эл",
            "Оренбургская область"
        ],
        "Желтый": [
            "Ямало-Ненецкий автономный округ",
            "Орловская область",
            "Чеченская Республика",
            "Калужская область",
            "Астраханская область",
            "Саратовская область",
            "Ханты-Мансийский автономный округ - Югра",
            "Республика Коми",
            "Республика Мордовия",
            "Республика Калмыкия",
            "Кабардино-Балкарская Республика",
            "Ставропольский край"
        ]
    }
    


    def __init__(self, inn):
        """Определяет тип контрагента и загружает данные."""
        self.inn = inn
        self.entity_type = self.detect_entity_type()
        self.data = self.fetch_data()

    def detect_entity_type(self):
        """Определяет, является ли ИНН ИП или ЮЛ."""
        if len(self.inn) == 12:
            return "ИП"
        elif len(self.inn) == 10:
            return "ЮЛ"
        else:
            raise ValueError("ИНН должен содержать 10 (ЮЛ) или 12 (ИП) цифр.")

    def fetch_data(self):
        """Получает данные из API."""
        url = self.BASE_URLS[self.entity_type]
        params = {"key": self.API_KEY, "inn": self.inn}

        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Ошибка {response.status_code}: {response.text}")


    def evaluate_risk_level(self, risk_name, value):
        """Определяет уровень риска."""
        if not value:
            return "Зеленый"
        if self.entity_type == "ИП":
            return "Красный" if risk_name in self.CRITICAL_RISKS_IP else "Желтый"
        return "Красный" if risk_name in self.CRITICAL_RISKS_UL else "Желтый"

    def check_region_risk(self):
        """Проверяет, находится ли контрагент в регионе с высокой кредитной нагрузкой."""
        region_name = self.data["data"].get("Регион", {}).get("Наим", "")

        if region_name in self.HIGH_RISK_REGIONS["Красный"]:
            return "Красный"
        elif region_name in self.HIGH_RISK_REGIONS["Желтый"]:
            return "Желтый"
        return "Зеленый"
    
    def analyze(self):
        """Определяет, является ли контрагент ИП или ЮЛ, и выполняет соответствующий анализ."""
        if self.entity_type == "ИП":
            risks_evaluated = self.analyze_ip()
        else:
            risks_evaluated = self.analyze_ul()

        print("🔎 **Риски контрагента**\n")
        for key, info in risks_evaluated.items():
            print(f"{key}: {info['Статус']} ({info['Уровень']})")

        print("\n--- Заключение ---\n")
        print(self.generate_risk_conclusion(risks_evaluated))
    

    def analyze_ip(self):
        """Анализирует риски для индивидуальных предпринимателей (ИП)."""
        region_risk = self.check_region_risk()
        region = self.data["data"].get("Регион", {}).get("Наим", "")
        reg_date = self.data["data"].get("ДатаРег", "2100-01-01")
        company_age = (datetime.date.today() - datetime.datetime.strptime(reg_date, "%Y-%m-%d").date()).days
        
        risks = {
            "1. Ограничение доступа к сведениям (ФНС)": self.data["data"].get("Статус", {}).get("ОгрДоступ", False),
            "2. Прекращение деятельности": bool(self.data["data"].get("Прекращ", {}).get("Дата")),
            "3. Включение в реестр недобросовестных поставщиков": self.data["data"].get("НедобПост", False),
            "4. Массовый руководитель": self.data["data"].get("МассРуковод", False),
            "5. Массовый учредитель": self.data["data"].get("МассУчред", False),
            "6. Записи в реестре банкротств (ЕФРСБ)": bool(self.data["data"].get("ЕФРСБ")),
            "7. Нарушения при получении господдержки": any(support.get("Наруш", False) for support in self.data["data"].get("ПоддержМСП", [])),
            "10. Компания существует менее 3 месяцев": company_age < 90,
            "11. Компания существует менее 1 года": company_age < 365,
            "12. Высокая доля просроченной задолженности в общей сумме задолженности по кредитам в регионе": region_risk == "Красный",
            "13. Повышенная доля просроченной задолженности в общей сумме задолженности по кредитам в регионе": region_risk == "Желтый",

        }

        risks_with_levels = {key: {"Статус": "Да" if value else "Нет", "Уровень": self.evaluate_risk_level(key, value)} for key, value in risks.items()}
        return risks_with_levels
    
    
    def analyze_ul(self):
        """Анализирует риски для ЮЛ."""
        region_risk = self.check_region_risk()
        region = self.data["data"].get("Регион", {}).get("Наим", "")
        reg_date = self.data["data"].get("ДатаРег", "2100-01-01")
        company_age = (datetime.date.today() - datetime.datetime.strptime(reg_date, "%Y-%m-%d").date()).days

        risks = {
            "1. Ограничение доступа к сведениям (ФНС)": self.data["data"].get("Статус", {}).get("ОгрДоступ", False),
            "2. Ликвидация компании": bool(self.data["data"].get("Ликвид", {}).get("Дата")),
            "3. Недостоверные сведения о юридическом адресе": self.data["data"].get("ЮрАдрес", {}).get("Недост", False),
            "4. Недостоверные сведения о руководителе": any(ru.get("Недост", False) for ru in self.data["data"].get("Руковод", [])),
            "5. Недостоверные сведения об учредителях": any(uch.get("Недост", False) for uch in self.data["data"].get("Учред", {}).get("ФЛ", [])),
            "6. Записи в реестре банкротств (ЕФРСБ)": bool(self.data["data"].get("ЕФРСБ")),
            "7. Включение в реестр недобросовестных поставщиков": self.data["data"].get("НедобПост", False),
            "8. Дисквалифицированные лица в руководстве": self.data["data"].get("ДисквЛица", False),
            "9. Включение в санкционные списки": self.data["data"].get("Санкции", False),
            "10. Нелегальная деятельность на финансовом рынке": self.data["data"].get("НелегалФин", False),
            "11. Массовый руководитель": self.data["data"].get("МассРуковод", False),
            "12. Массовый учредитель": self.data["data"].get("МассУчред", False),
            "15. Наличие задолженности по налогам": bool(self.data["data"].get("Налоги", {}).get("СумНедоим", 0)),
            "16. Высокая доля просроченной задолженности в общей сумме задолженности по кредитам в регионе": region_risk == "Красный",
            "17. Повышенная доля просроченной задолженности в общей сумме задолженности по кредитам в регионе": region_risk == "Желтый",
            "18. Компания существует менее 3 месяцев": company_age < 90,
            "19. Компания существует менее 1 года": company_age < 365,
            "20. Нарушения при получении господдержки": any(support.get("Наруш", False) for support in self.data["data"].get("ПоддержМСП", [])),

        }

        risks_with_levels = {key: {"Статус": "Да" if value else "Нет", "Уровень": self.evaluate_risk_level(key, value)} for key, value in risks.items()}
        return risks_with_levels

    def generate_risk_conclusion(self, risks_evaluated):
        """Генерирует заключение по рискам."""
        critical_issues = []
        non_critical_issues = []

        for risk, details in risks_evaluated.items():
            if details["Уровень"] == "Красный":
                critical_issues.append(risk)
            elif details["Уровень"] == "Желтый":
                non_critical_issues.append(risk)

        if not critical_issues and not non_critical_issues:
            return "✅ Все риски зеленые! Контрагент ведет прозрачную деятельность."

        elif critical_issues:
            message = "🚨 Обнаружены **критические риски**:\n"
            for issue in critical_issues:
                message += f"- **{issue}** - необходимо срочно решать этот вопрос!\n"
            message += "\n⚠️ Рекомендуется тщательно проверить деятельность контрагента и избегать сотрудничества без дополнительной проверки."
            return message

        elif non_critical_issues:
            message = "⚠️ Обнаружены **некритичные (желтые) риски**:\n"
            for issue in non_critical_issues:
                message += f"- **{issue}** - стоит обратить внимание, но не является критичным.\n"
            return message



In [14]:
inn = "772777042905"  
analyzer = RiskAnalyzer(inn)
analyzer.analyze()

🔎 **Риски контрагента**

1. Ограничение доступа к сведениям (ФНС): Нет (Зеленый)
2. Прекращение деятельности: Да (Красный)
3. Включение в реестр недобросовестных поставщиков: Нет (Зеленый)
4. Массовый руководитель: Нет (Зеленый)
5. Массовый учредитель: Нет (Зеленый)
6. Записи в реестре банкротств (ЕФРСБ): Нет (Зеленый)
7. Нарушения при получении господдержки: Нет (Зеленый)
10. Компания существует менее 3 месяцев: Нет (Зеленый)
11. Компания существует менее 1 года: Нет (Зеленый)
12. Высокая доля просроченной задолженности в общей сумме задолженности по кредитам в регионе: Нет (Зеленый)
13. Повышенная доля просроченной задолженности в общей сумме задолженности по кредитам в регионе: Нет (Зеленый)

--- Заключение ---

🚨 Обнаружены **критические риски**:
- **2. Прекращение деятельности** - необходимо срочно решать этот вопрос!

⚠️ Рекомендуется тщательно проверить деятельность контрагента и избегать сотрудничества без дополнительной проверки.


In [15]:
import requests
import pandas as pd


def load_data(table, inn):
    
    API_KEY = "AhmhPqXQDWyWgqOY"

    # Базовый URL API
    BASE_URL = f"https://api.checko.ru/v2/{table}"


    params = {
        "key": API_KEY,  
        "inn": inn       
    }

    # Выполняем GET-запрос
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()  
        return data
    else:
        print(f"Ошибка {response.status_code}: {response.text}")



company_profits = {}
company_aktiv = {}
company_credit_capacity = {}
def analyze_financials(data, inn):
    """
    Анализирует финансовые данные компании и выявляет риски.
    """
    if not data or "data" not in data:
        return "🚨 Финансовые данные отсутствуют."

    years = sorted(data["data"].keys(), reverse=True)
    if not years:
        return "🚨 Нет данных по годам в финансовой отчетности."

    latest_year = years[0]  
    latest_report = data["data"][latest_year]

    # Определяем ключевые показатели
    revenue = latest_report.get('2110', 0)  # Выручка
    net_profit = latest_report.get('2400', 0)  # Чистая прибыль (убыток)
    accounts_receivable = latest_report.get('1230', 0)  # Дебиторская задолженность
    short_term_loans = latest_report.get('1510', 0)  # Краткосрочные кредиты
    total_loans = latest_report.get('1500', 0)  # Общая сумма заемных обязательств
    authorized_capital = latest_report.get('1310', 0)  # Уставной капитал
    total_capital = latest_report.get('1300', 0)  # Всего капитал и резервы (используем для анализа уменьшения капитала)
    reorganization = latest_report.get('2510', 0)  # Реорганизация юридического лица
    aktiv = latest_report.get('1600', 0)  # валюта баланса

    company_profits[inn] = net_profit
    company_aktiv[inn] = aktiv

    risk_messages = []

    if net_profit < 0:
        risk_messages.append(f"🚨 **Красный риск**: В {latest_year} году убыток составил {net_profit:,} руб.")
        
    elif net_profit == 0:
        risk_messages.append(f"🚨 **Красный риск**: В {latest_year} году прибыль составила {net_profit:,} руб.")
        
    elif net_profit < (0.1 * revenue):  
        risk_messages.append(f"⚠️ **Желтый риск**: Низкая маржинальность ({net_profit:,} руб.) в {latest_year} году.")
        
    elif years[1] in data["data"]:  
        prev_year_revenue = data["data"][years[1]].get('2110', 0)
        if prev_year_revenue > 0 and revenue <= 0.5 * prev_year_revenue:
            risk_messages.append(f"🚨 **Красный риск**: Выручка снизилась в два раза и более в сравнение с прошлым годом ({revenue:,} руб., в прошлом периоде ({prev_year_revenue})) руб.")
            
    elif years[1] in data["data"]:  
        prev_year_revenue = data["data"][years[1]].get('2110', 0)
        if prev_year_revenue > 0 and revenue <= 0.8 * prev_year_revenue and revenue > 0.5 * prev_year_revenue:
            risk_messages.append(f"⚠️ **Желтый риск**: Выручка снизилась более чем на 20% ({revenue:,} руб.)")


    # 2. Дебиторская задолженность
    if revenue > 0:
        ar_ratio = accounts_receivable / revenue
        if ar_ratio >= 0.5:
            risk_messages.append(f"🚨 **Красный риск**: Дебиторская задолженность превышает 50% выручки ({ar_ratio:.1%})")
        elif ar_ratio > 0.2 and ar_ratio < 0.5:
            risk_messages.append(f"⚠️ **Желтый риск**: Дебиторская задолженность составляет 20-50% выручки ({ar_ratio:.1%})")

    # 3. Кредитная нагрузка
    if total_loans > 0:
        short_term_ratio = total_loans / revenue
        red_zone_threshold = revenue * 0.4
        remaining_credit_capacity = max(red_zone_threshold - total_loans, 0)
        company_credit_capacity[inn] = remaining_credit_capacity
        if short_term_ratio >= 0.4:
            risk_messages.append("🚨 **Красный риск**: Краткосрочные кредиты > 40% от выручки.")
        elif short_term_ratio >= 0.2 and short_term_ratio < 0.4 :
            risk_messages.append("⚠️ **Желтый риск**: Краткосрочные кредиты составляют 20-40% выручки.")


    # 4. Уменьшение капитала
    if years[1] in data["data"]:  # Проверяем данные за предыдущий год
        prev_total_capital = data["data"][years[1]].get(1300, 0)
        if prev_total_capital > 0 and total_capital < 0.9 * prev_total_capital:  # Капитал уменьшился на 10% и более
            risk_messages.append(f"🚨 **Красный риск**: Капитал и резервы уменьшились более чем на 10% ({total_capital:,} руб.).")

    # 5. Реорганизация юридического лица
    if reorganization > 0:
        risk_messages.append(f"⚠️ **Желтый риск**: В {latest_year} году была реорганизация юридического лица.")

    if not risk_messages:
        return f"✅ Контрагент стабилен: прибыль за {latest_year} год составила {net_profit:,} руб."

    return "\n".join(risk_messages)


def analyze_enforcements(data, inn):
    """
    Анализирует исполнительные производства с учетом ЮЛ и ИП.
    """
    if not data or "data" not in data or "Записи" not in data["data"]:
        return "✅ У контрагента нет исполнительных производств."

    enforcements = data["data"]["Записи"]
    total_cases = len(enforcements)  
    total_debt = sum(case.get("СумДолг", 0) for case in enforcements)  
    outstanding_debt = sum(case.get("ОстЗадолж", 0) for case in enforcements)  

    is_ip = len(inn) == 12  

    net_profit = company_profits.get(inn, None) 
    aktiv = company_aktiv.get(inn, None) 

    risk_messages = []

    # 1. Красный риск для ЮЛ: задолженность >= 20% от валюты баланса
    if not is_ip and net_profit and (total_debt >= 0.2 * aktiv):
        risk_messages.append(
            f"🚨 **Красный риск**: {total_cases} дел на сумму {total_debt:,} руб., "
            f"из них задолженность {total_debt / aktiv:.1%} ({outstanding_debt:,} руб.) от валюты баланса!"
        )
        
    elif net_profit is None or float(net_profit) == 0:
        if total_cases > 0:
            risk_messages.append(
                f"🚨 **Красный риск**: {total_cases} дел на сумму {total_debt:,} руб. (прибыль отсуствует), "

            )        
        
        
        
    # 3. Желтый риск для ЮЛ: задолженность от 10% до 20% от валюты баланса
    elif not is_ip and net_profit and total_debt < 0.2 * aktiv and total_debt >= 0.1 * aktiv:
        risk_messages.append(
            f"⚠️ **Желтый риск**: {total_cases} дел на сумму {total_debt:,} руб., "
            f"из них задолженность {total_debt / aktiv:.1%} ({outstanding_debt:,} руб.) от валюты баланса!."
        )


    # Итоговое сообщение
    if not risk_messages:
        return "✅ У контрагента нет значимых рисков по исполнительным производствам."

    return "\n".join(risk_messages)

def analyze_legal_cases(data, inn, user_revenue):
    """
    Анализирует арбитражные дела с учетом ЮЛ и ИП.
    """
    if not data or "data" not in data or "Записи" not in data["data"]:
        return "✅ В арбитражных судах РФ не найдено судебных дел с участием контрагента."

    cases = data["data"]["Записи"]
    total_cases = len(cases)  
    total_claims = sum(case.get("СуммИск", 0) for case in cases)  

    plaintiff_cases = sum(1 for case in cases if any(p.get("ИНН") == inn for p in case.get("Ист", [])))
    defendant_cases = sum(1 for case in cases if any(d.get("ИНН") == inn for d in case.get("Ответ", [])))

    is_ip = len(inn) == 12 

    net_profit = company_profits.get(inn, None) 
    aktiv = company_aktiv.get(inn, None) 

    risk_messages = []
    

    # 1. Красный риск для ЮЛ: сумма исков > 20% от валюты баланса.
    if not is_ip and net_profit and (total_claims >= 0.2 * aktiv):
        risk_messages.append(
            f"🚨 **Красный риск**: В арбитражных судах РФ было рассмотрено {total_cases} дел "
            f"(истец: {plaintiff_cases}, ответчик: {defendant_cases}). "
            f"Общая сумма исков: {total_claims:,} руб. ({total_claims / aktiv:.1%} от валюты балаанса)."
        )
        
        
    elif not is_ip and (net_profit is None or float(net_profit) == 0):
        if total_cases > 0:
            risk_messages.append(
                f"🚨 **Красный риск**: В арбитражных судах РФ было рассмотрено {total_cases} дел "
                f"(истец: {plaintiff_cases}, ответчик: {defendant_cases}). "
                f"Общая сумма исков: {total_claims:,} руб. (прибыль отсутствует)."
            )

    # 2. Красный риск для ИП: сумма исков > 0.2 от выручки.
    elif is_ip and total_claims >= 0.2 * user_revenue:
        risk_messages.append(
            f"🚨 **Красный риск**: В арбитражных судах РФ было рассмотрено {total_cases} дел "
            f"(истец: {plaintiff_cases}, ответчик: {defendant_cases}). "
            f"Общая сумма исков превышает 20% от выручки."
        )

    # 3. Желтый риск для ЮЛ: сумма исков > 10% до 20 % от валюты баланса
    elif not is_ip and net_profit and total_claims > 0.1 * aktiv and total_claims < 0.2 * aktiv :
        risk_messages.append(
            f"⚠️ **Желтый риск**: В арбитражных судах РФ было рассмотрено {total_cases} дел "
            f"(истец: {plaintiff_cases}, ответчик: {defendant_cases}). "
            f"Общая сумма исков: {total_claims:,} руб. ({total_claims / net_profit:.1%} от валюты баланса)."
        )

    # 4. Желтый риск для ИП: сумма исков от 0.1 до 0.2 от выручки.
    elif is_ip and total_claims >= 0.1 * user_revenue and total_claims < 0.2 * user_revenue:
        risk_messages.append(
            f"⚠️ **Желтый риск**: В арбитражных судах РФ было рассмотрено {total_cases} дел "
            f"(истец: {plaintiff_cases}, ответчик: {defendant_cases}). "
            f"Общая сумма исков превышает 10% от выручки."
        )

    # Итоговое сообщение
    if not risk_messages:
        return "✅ В арбитражных судах РФ не найдено значимых судебных рисков для контрагента."

    return "\n".join(risk_messages)

def get_loan_amount():
        while True:
            try:
                amount = float(input("Введите объем кредита, который вы хотите выплатить: "))
                if amount <= 0:
                    print("Сумма должна быть положительным числом. Попробуйте снова.")
                else:
                    print(f"Вы указали сумму кредита: {amount:,.0f} рублей.".replace(",", " "))
                    return amount
            except ValueError:
                print("Пожалуйста, введите числовое значение.")

def get_revenue_input():
        while True:
            try:
                revenue = float(input("Введите годовой оборот (выручку): "))
                if revenue <= 0:
                    print("Выручка должна быть положительной. Попробуйте снова.")
                else:
                    print(f"Вы указали выручку: {revenue:,.0f} рублей.".replace(",", " "))
                    return revenue
            except ValueError:
                print("Пожалуйста, введите числовое значение.")



In [16]:
def analyze_company(inn):
    """
    Запускает анализ проверок, финансов и исполнительных производств по ИНН.
    """
    results = []
    
    is_ip = len(inn) == 12
    
    user_loan_amount = get_loan_amount()
    user_revenue = None
    if is_ip:
        user_revenue = get_revenue_input()
        
        
    # 🔍 Проверка кредитной нагрузки на основании пользовательского ввода
    loan_risk_msg = ""
    if is_ip and user_revenue:
        loan_ratio = user_loan_amount / user_revenue
        if loan_ratio >= 0.2:
            loan_risk_msg = f"🚨 **Красный риск**: Объем кредита составляет {loan_ratio:.1%} от выручки ИП."
        elif loan_ratio >= 0.1:
            loan_risk_msg = f"⚠️ **Желтый риск**: Объем кредита составляет {loan_ratio:.1%} от выручки ИП."
    else:
        allowed_credit = company_credit_capacity.get(inn, None)
        if allowed_credit is not None and user_loan_amount > allowed_credit:
            loan_risk_msg = (
                f"🚨 **Красный риск**: Запрошенный объем кредита ({user_loan_amount:,.0f} руб.) превышает допустимую "
                f"нагрузку ({allowed_credit:,.0f} руб.).".replace(",", " ")
            )

    if loan_risk_msg:
        results.append(f"### Кредитная нагрузка по введенным данным\n{loan_risk_msg}\n")
    
    # Анализ финансовых показателей
    
    if not is_ip:
        financials_data = load_data("finances", inn)
        financials_analysis = analyze_financials(financials_data, inn)
        results.append(f"### Анализ финансов\n{financials_analysis}\n")
    
    # Анализ исполнительных производств
    enforcements_data = load_data("enforcements", inn)
    enforcements_analysis = analyze_enforcements(enforcements_data, inn)
    results.append(f"### Анализ исполнительных производств\n{enforcements_analysis}\n")
    # Анализ арбитражных дел
    arbitrazh_data = load_data("legal-cases", inn)
    arbitrazh_analysis = analyze_legal_cases(arbitrazh_data, inn, user_revenue)
    results.append(f"### Анализ арбитражных дел\n{arbitrazh_analysis}\n")
    # Итоговый отчет
    report = "\n".join(results)
    return report

In [17]:
inn = "3702067805"  
print(analyze_company(inn))

Введите объем кредита, который вы хотите выплатить: 100000000
Вы указали сумму кредита: 100 000 000 рублей.
### Анализ финансов
⚠️ **Желтый риск**: Низкая маржинальность (170,771,000 руб.) в 2024 году.
⚠️ **Желтый риск**: Дебиторская задолженность составляет 20-50% выручки (21.0%)
⚠️ **Желтый риск**: Краткосрочные кредиты составляют 20-40% выручки.

### Анализ исполнительных производств
✅ У контрагента нет значимых рисков по исполнительным производствам.

### Анализ арбитражных дел
✅ В арбитражных судах РФ не найдено значимых судебных рисков для контрагента.

